# Demo Notebook for using the Reboot Recommendation Engine

In [ ]:
#@title Install Python Packages

# !pip install git+https://github.com/RebootMotion/reboot-toolkit.git@v2.4.1#egg=reboot_toolkit > /dev/null
# !pip install git+https://github.com/RebootMotion/mlb-statsapi.git@v1.1.0#egg=mlb_statsapi > /dev/null
# !echo "Done Installing"

In [ ]:
%cd ..

In [4]:
#@title Import Statements

import reboot_toolkit as rtk
from reboot_toolkit import MovementType, setup_aws, MovementType, MocapType, Handedness, FileType, S3Metadata, PlayerMetadata
import os
import pandas as pd
import ipywidgets as widgets

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 999999em; }</style>"))

In [3]:
password_out = widgets.Output(layout={'border': '1px solid black'})
with password_out:
    boto3_session = setup_aws(org_id="org-mlbbiomech", aws_default_region="us-west-1")
    
display(password_out)

Input org_id here (or input empty string to use org-mlbbiomech):········


In [ ]:
w1 = widgets.Dropdown(options=[e.value for e in MocapType], value=MocapType.HAWKEYE_HFR.value, description="MocapType: ")
w2 = widgets.Dropdown(options=[e.value for e in MovementType], value=MovementType.BASEBALL_PITCHING.value, description="MovementType: ")
w3 = widgets.Dropdown(options=[e.value for e in Handedness], value=Handedness.LEFT.value, description="Handedness: ")
w4 = widgets.Dropdown(options=[e.value for e in FileType], value=FileType.METRICS_BASEBALL_PITCHING_V2.value, description="FileType: ")

file_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
file_loading_out = widgets.Output()
file_in = widgets.VBox([w1, w2, w3, w4, widgets.HBox([file_button, file_loading_out])])
file_out = widgets.Output(layout={'border': '1px solid black', 'max_height': '350px', 'overflow': 'scroll'})

data_section = widgets.Output()
analysis_section = widgets.Output()

def file_f(_):
    with file_loading_out:
        print("Loading...")
    mocap_type = MocapType(w1.value)
    movement_type = MovementType(w2.value)
    handedness = Handedness(w3.value)

    s3_metadata = S3Metadata(
        org_id=os.environ['ORG_ID'],
        mocap_types=[mocap_type],
        movement_type=movement_type,
        handedness=handedness,
        file_type=FileType(w4.value),
    )

    with file_out:
        file_out.clear_output()
        s3_df = rtk.download_s3_summary_df(s3_metadata)
    file_loading_out.clear_output()

    # Set up next section
    primary_segment_widget = rtk.create_interactive_widget(s3_df)
    data_button = widgets.Button(
        description='Submit',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Submit',
        icon='check' # (FontAwesome names without the `fa-` prefix)
    )
    data_in = widgets.VBox([primary_segment_widget, data_button])
    data_out = widgets.Output(layout={'border': '1px solid black'})

    def data_f(_):
        with data_out:
            data_out.clear_output()
            print(primary_segment_widget.result)

        analysis_button = widgets.widgets.Button(
            description='Run Analysis',
            disabled=False,
            button_style='success', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Sun Analysis',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )
        analysis_loading_out = widgets.Output()
        analysis_out = widgets.Output(layout={'max_height': '350px', 'overflow': 'scroll'})
        analysis_result = widgets.Output()
        def analysis_f(_):
            with analysis_loading_out:
                print("Loading...")
            with analysis_out:
                analysis_out.clear_output()
                primary_segment_label = 'Primary Segment'

                assert primary_segment_widget.result is not None
                primary_analysis_segment = PlayerMetadata(
                    org_player_ids=primary_segment_widget.result["org_player_ids"],
                    session_dates=primary_segment_widget.result["session_dates"],
                    session_nums=primary_segment_widget.result["session_nums"],
                    session_date_start=primary_segment_widget.result["session_date_start"],
                    session_date_end=primary_segment_widget.result["session_date_end"],
                    year=primary_segment_widget.result["year"],
                    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
                    s3_metadata=s3_metadata,
                )

                primary_segment_summary_df = rtk.filter_s3_summary_df(primary_analysis_segment, s3_df)
                prim_available_s3_keys = rtk.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)
                primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())
                primary_segment_data_df = rtk.merge_data_df_with_s3_keys(primary_segment_data_df, prim_available_s3_keys).sort_values(by=['session_date', 'movement_num'])
                df = rtk.recommendation(boto3_session, primary_segment_data_df, movement_type, mocap_type, handedness)

            with analysis_result:
                analysis_result.clear_output()
                fig = rtk.recommendation_violin_plot(boto3_session, primary_segment_data_df, movement_type, mocap_type, handedness, num_features=5)
                fig.show()
            analysis_loading_out.clear_output()
        analysis_button.on_click(analysis_f)

        with analysis_section:
            display(widgets.HBox([analysis_button, analysis_loading_out]), analysis_out, analysis_result)

    data_button.on_click(data_f)
    with data_section:
        data_section.clear_output()
        display(data_in, data_out)

file_button.on_click(file_f)

display(file_in, file_out, data_section, analysis_section)